# Centralized MINLP Optimization (Julia, JuMP)

This notebook implements the centralized epidemic optimization model (see page 1378 of the uploaded paper).

- Objective: **minimize total new infections**
- Decision variables: NPI levels (1–5) per week
- Budget constraint: severity cost normalized per person per week
- Strategy: **Centralized** (all 25 regions combined as one country)

We sweep over different budgets and compare optimal outcomes.

In [ ]:
using JuMP, Juniper, Ipopt, GLPK, DataFrames, Plots

# ---------------- Parameters ----------------
T = 8                      # planning horizon
capacity = [93, 42, 22, 49, 14, 15, 31, 19, 18, 20, 5 , 6, 6, 4, 18, 27, 11,
            43, 14, 28, 14, 26, 11, 28, 18]
P = [2480000,2443000,1296000,1501000,530000,780000,1147000,873000,676000,
     626000,133000,114000,194000,98000,590000,752000,441000,1743000,849000,954000,
     440000,895000,505000,1190000,898000]
beta_vec = [1.0,1.0,0.8,1.0,0.7,0.7,0.8,0.7,0.7,0.7,0.7,0.7,0.7,0.7,
            0.7,0.7,0.7,1.0,0.7,0.7,0.7,0.7,0.7,1.0,0.7]

# centralized aggregated values
P_total = sum(P)
beta_central = mean(beta_vec)
capacity_total = sum(capacity)

# epidemic params
phi = 1.0; mu = 0.4; gamma = 0.1
rho = 0.1; epsilon = 0.18
alpha = [0.12,0.20,0.27,0.35,0.52]
init_infected_frac = 0.0001
critInf_frac = 0.000001

# severity costs
sev_cost = [0.014,0.04,0.073,0.123,0.27]

In [ ]:
function solve_for_budget(B_norm)
    model = Model(Juniper.Optimizer)
    set_optimizer_attribute(model, "nl_solver", optimizer_with_attributes(Ipopt.Optimizer))
    set_optimizer_attribute(model, "mip_solver", optimizer_with_attributes(GLPK.Optimizer))

    # Decision variables
    @variable(model, 1 <= lvl[1:T] <= 5, Int)

    # State variables
    @variable(model, I[0:T] >= 0)
    @variable(model, C[0:T] >= 0)
    @variable(model, D[0:T] >= 0)
    @variable(model, newInf[1:T] >= 0)

    # Initial conditions
    @constraint(model, I[0] == P_total * init_infected_frac)
    @constraint(model, C[0] == P_total * critInf_frac)
    @constraint(model, D[0] == 0)

    # Epidemic dynamics
    for t in 1:T
        @NLconstraint(model, newInf[t] == beta_central * (1 - alpha[lvl[t]]) * (1 - rho*epsilon) * I[t-1])
        @NLconstraint(model, I[t] == I[t-1] + newInf[t] - mu*(1-rho)*I[t-1] - rho*phi*I[t-1])
        @NLconstraint(model, C[t] == (1-gamma)*C[t-1] + phi*I[t-1])
        @NLconstraint(model, D[t] == D[t-1] + 0.02*C[t-1])
    end

    # Budget constraint
    b_agg = B_norm * P_total * T
    @constraint(model, sum(P_total * sev_cost[lvl[t]] for t in 1:T) <= b_agg)

    # Objective
    @objective(model, Min, sum(newInf[t] for t in 1:T))

    optimize!(model)

    return (
        budget = B_norm,
        infections = objective_value(model),
        deaths = value(D[T]),
        seq = value.(lvl)
    )
end

In [ ]:
# Sweep over budgets and collect results
budgets = 0.05:0.05:0.30
results = [solve_for_budget(b) for b in budgets]

# Display results
for r in results
    println("Budget=", r.budget, " Infections=", r.infections,
            " Deaths=", r.deaths, " Sequence=", r.seq)
end

In [ ]:
# Convert results into DataFrame and plot
df = DataFrame(budget=[r.budget for r in results],
               infections=[r.infections for r in results],
               deaths=[r.deaths for r in results])

println(df)

plot(df.budget, df.infections, marker=:circle, xlabel="Budget", ylabel="Infections",
     title="Infections vs Budget", legend=false)